In [1]:
using Pkg
#Pkg.add("CSV")
#Pkg.add("Printf")
#Pkg.add("DataFrames")
#Pkg.add("Tables")
#Pkg.add("Statistics")
#Pkg.add("Query")
#Pkg.add("CUDA")
#Pkg.add("Dates")
#Pkg.add("Printf")
#Pkg.add("BenchmarkTools")
using CSV
using DataFrames
using Printf
using PrettyTables
#using Tables
#using Statistics
#using Query
#using CUDA
using Dates
#using Printf
#using Base.Threads
using BenchmarkTools

In [2]:
df_ratings = CSV.read("/Users/sebastianmerino/Documents/Learning/Diplomado IA/Movie Lens Project/ratings.csv", DataFrame)
df_movies = CSV.read("/Users/sebastianmerino/Documents/Learning/Diplomado IA/Movie Lens Project/movies.csv", DataFrame)
df_movies_reduced = select(df_movies, [:movieId, :genres])

Row,movieId,genres
,Int64,String
1,1,Adventure|Animation|Children|Comedy|Fantasy
2,2,Adventure|Children|Fantasy
3,3,Comedy|Romance
4,4,Comedy|Drama|Romance
5,5,Comedy
6,6,Action|Crime|Thriller
7,7,Comedy|Romance
8,8,Adventure|Children
9,9,Action


In [3]:
# Contando el numero de registros del archivo
num_rows_ratings = nrow(df_ratings)
num_rows_movies = nrow(df_movies)

print("Filas del archivo ratings: ", num_rows_ratings, ", Filas del archivo movies: ", num_rows_movies)

Filas del archivo ratings: 25000095, Filas del archivo movies: 62423

In [4]:
# Numero de Chunks
n = 10
filas_archivo = num_rows_ratings/n
Num_archivos = num_rows_ratings/filas_archivo

# Definimos el path de salida 
path = "/Users/sebastianmerino/Documents/Learning/Diplomado IA/Movie Lens Project/Splitted File/"

# Definimos la función que nos divide el archivo en chunks
function file_splitter(n=10)
    println("Dividiendo el file en $n chunks")
    # Capturamos la hora de inicio
    start_time = now()

    for i in 1:n
        inicio = (i-1)*filas_archivo+1
        if i < n
            final = i*filas_archivo
        else i=n
            final = num_rows_ratings
        end
        inicio = ceil(Int, inicio)
        final = ceil(Int, final)
        DF_aux = df_ratings[Int(inicio):Int(final), :]
        CSV.write(joinpath(path,"Archivo_$(i).csv"),DF_aux)
        println("Iteración No: ",i)
        @printf("%d\n",inicio)
        @printf("%d\n",final)
    end
    # Capturamos la hora de fin
    end_time = now()
    # Calculamos el tiempo de ejecución
    tiempo_ejecucion = end_time - start_time
    println("Tiempo de ejecución: ", tiempo_ejecucion)
end

file_splitter (generic function with 2 methods)

In [5]:
# Ejecutamos la función que divide ratings
file_splitter(n)

Dividiendo el file en 10 chunks
Iteración No: 1
1
2500010
Iteración No: 2
2500011
5000019
Iteración No: 3
5000020
7500029
Iteración No: 4
7500030
10000038
Iteración No: 5
10000039
12500048
Iteración No: 6
12500049
15000057
Iteración No: 7
15000058
17500067
Iteración No: 8
17500068
20000076
Iteración No: 9
20000077
22500086
Iteración No: 10
22500087
25000095
Tiempo de ejecución: 9608 milliseconds


In [6]:
# Creamos la función que se encargará de realizar la lectura de los csv
function read_separated_files(base_name::String, n::Int)
    # Pre-identificamos los archivos que se van a leer en un diccionario
    dataframes = Dict{String, DataFrame}()

    # Leemos los csv
    for i in 1:n
        file_name = joinpath(path, "$(base_name)_$(i).csv")
        var_name = "$(base_name)_$(i)"
        dataframes[var_name] = CSV.read(file_name, DataFrame)
    end

    return dataframes
end

read_separated_files (generic function with 1 method)

In [7]:
# Evaluamos la función
base_name = "Archivo"
n_files = 10

read_ratings = read_separated_files(base_name, n_files)

Dict{String, DataFrame} with 10 entries:
  "Archivo_7"  => 2500010×4 DataFrame…
  "Archivo_9"  => 2500010×4 DataFrame…
  "Archivo_2"  => 2500009×4 DataFrame…
  "Archivo_6"  => 2500009×4 DataFrame…
  "Archivo_5"  => 2500010×4 DataFrame…
  "Archivo_8"  => 2500009×4 DataFrame…
  "Archivo_4"  => 2500009×4 DataFrame…
  "Archivo_1"  => 2500010×4 DataFrame…
  "Archivo_3"  => 2500010×4 DataFrame…
  "Archivo_10" => 2500009×4 DataFrame…

In [8]:
# Verificamos que la lectura de los archivos divididos tenga el mismo tamaño que la lectura del archivo ratings
# Inicializamos la variable del total de filas
total_rows = 0

for i in 1:10
    key = "Archivo_$i"  # La key para cada dataframe
    if haskey(read_ratings, key)  # Revisamos si existe en el diccionario
        total_rows += nrow(read_ratings[key])  # Sumamos las filas
    else
        println("Key $key no existe en el diccionario.")
    end
end

println("El total de filas de los 10 files divididos: $total_rows")

El total de filas de los 10 files divididos: 25000095


In [9]:
# Contamos las calificaciones por genero
# Contando los generos
kg = ["Action", "Adventure", "Animation", "Children", "Comedy", "Crime", "Documentary",
		"Drama", "Fantasy", "Film-Noir", "Horror", "IMAX", "Musical", "Mystery", "Romance",
		"Sci-Fi", "Thriller", "War", "Western", "(no genres listed)"]
ng = length(kg)

nF = 10 # Numero de particiones
#ra = zeros(ng, nF)
#ca = zeros(Int, ng, nF)

10

In [ ]:
# Función para procesar cada archivo ratings y acumular resultados por género
#function process_ratings(ng::Int, genres::Vector{String}, df_movies::DataFrame, df_ratings::DataFrame)
#    # Inicializar un vector vacío de diccionarios
#    genre_results = []
#
    # Hacer un inner join entre movies y ratings
#    joined_df = innerjoin(df_movies, df_ratings, on=:movieId)

    # Calcular sumas y conteos por cada género
#    for i in 1:ng
        # Filtrar las filas que contienen el género actual
#        genre_rows = joined_df[occursin.(genres[i], joined_df.genres), :]
#
        # Verificar que se hayan encontrado filas con ese género
#       count_genre = nrow(genre_rows)  # Número de ratings para el género
#        sum_ratings = sum(genre_rows.rating)  # Suma de las calificaciones para el género
#
        # Crear un diccionario con los resultados para este género
#        result = Dict(
#            "genre" => genres[i],
#            "rating_sum" => sum_ratings,
#            "rating_count" => count_genre
#        )
#
        # Agregar el diccionario al vector de resultados
#        push!(genre_results, result)
#    end
#
#    return genre_results
#end
#
# Generar el resultado con 5 géneros
#genre_results = process_ratings(ng, kg, df_movies, df_ratings)
#
# Mostrar los resultados
#println(genre_results)

In [ ]:
# genre_results

In [10]:
# Función que calcula el número de calificaciones por género
function calcular_generos(dfr, dfm)

    start_time = now()
    # Realizar el inner join entre ratings y movies
    full_rating = innerjoin(dfr, dfm, on=:movieId)

    # Función auxiliar para dividir los géneros y contar
    contar_generos(fila, contador) = begin
        # Dividir los géneros en la fila actual
        generos = split(fila.genres, "|")

        # Actualizar el contador para cada género
        for genero in generos
            contador[genero] = get(contador, genero, 0) + 1
        end
    end

    # Crear un contador vacío para almacenar los géneros
    contador = Dict{String, Int}()

    # Aplicar la función contar_generos a cada fila de la tabla
    map(fila -> contar_generos(fila, contador), eachrow(full_rating))

    end_time = now()
    println("Tiempo de ejecución: ", end_time - start_time, " segundos")
    return contador
end

calcular_generos (generic function with 1 method)

In [11]:
calcular_generos(df_ratings,df_movies)

Tiempo de ejecución: 12613 milliseconds segundos


Dict{String, Int64} with 20 entries:
  "War"                => 1267346
  "Documentary"        => 322449
  "Action"             => 7446918
  "Adventure"          => 5832424
  "Mystery"            => 2010995
  "Children"           => 2124258
  "Thriller"           => 6763272
  "Sci-Fi"             => 4325740
  "Comedy"             => 8926230
  "(no genres listed)" => 26627
  "IMAX"               => 1063279
  "Fantasy"            => 2831585
  "Drama"              => 10962833
  "Animation"          => 1630987
  "Western"            => 483731
  "Horror"             => 1892183
  "Romance"            => 4497291
  "Film-Noir"          => 247227
  "Crime"              => 4190259
  "Musical"            => 964252

In [12]:
# Arreglo para almacenar los resultados de cada hilo
results = Vector{Dict{String, Int}}(undef, 10) 

st = now()
# Inicializar los hilos
Threads.@threads for i in 1:10
    key = "Archivo_$(i)"  # Create the key dynamically
    if haskey(read_ratings, key)
        results[i] = calcular_generos(read_ratings[key],df_movies)  # Print the value associated with the key
    else
        results[i] = Dict{String, Int}()  # Si no existe la clave, crea un diccionario vacío
    end
end

en = now()
println("Tiempo de ejecución: ", en - st, " segundos")

Tiempo de ejecución: 1177 milliseconds segundos
Tiempo de ejecución: 1131 milliseconds segundos
Tiempo de ejecución: 1071 milliseconds segundos
Tiempo de ejecución: 1085 milliseconds segundos
Tiempo de ejecución: 1318 milliseconds segundos
Tiempo de ejecución: 1176 milliseconds segundos
Tiempo de ejecución: 1122 milliseconds segundos
Tiempo de ejecución: 1328 milliseconds segundos
Tiempo de ejecución: 1146 milliseconds segundos
Tiempo de ejecución: 1125 milliseconds segundos
Tiempo de ejecución: 11982 milliseconds segundos


In [13]:
# Inicializar un diccionario para la suma total de géneros
suma_total = Dict{String, Int}()

# Sumar los resultados de todos los diccionarios en results
for diccionario in results
    if diccionario !== nothing  # Verificar que el diccionario no sea `nothing`
        for (genero, conteo) in diccionario
            suma_total[genero] = get(suma_total, genero, 0) + conteo
        end
    end
end

# Mostrar la suma total de géneros
println(suma_total)

Dict("War" => 1267346, "Documentary" => 322449, "Action" => 7446918, "Adventure" => 5832424, "Mystery" => 2010995, "Children" => 2124258, "Thriller" => 6763272, "Sci-Fi" => 4325740, "Comedy" => 8926230, "(no genres listed)" => 26627, "IMAX" => 1063279, "Fantasy" => 2831585, "Drama" => 10962833, "Animation" => 1630987, "Western" => 483731, "Horror" => 1892183, "Romance" => 4497291, "Film-Noir" => 247227, "Crime" => 4190259, "Musical" => 964252)


In [14]:
# Sort by keys alphabetically in descending order
sorted_by_key_desc = sort(collect(suma_total), by=x->x[1], rev=false)

# Print sorted results
for (key, value) in sorted_by_key_desc
    println("$key: $value")
end

(no genres listed): 26627
Action: 7446918
Adventure: 5832424
Animation: 1630987
Children: 2124258
Comedy: 8926230
Crime: 4190259
Documentary: 322449
Drama: 10962833
Fantasy: 2831585
Film-Noir: 247227
Horror: 1892183
IMAX: 1063279
Musical: 964252
Mystery: 2010995
Romance: 4497291
Sci-Fi: 4325740
Thriller: 6763272
War: 1267346
Western: 483731
